# Conversion of 1QIsaa to Text-Fabric

Conversion of 1QIsaa data to TF format. I work towards a full ETCBC encoding on the levels of characters, words, phrases, and clauses, for all the DSS.

In [37]:
import re
import collections
import json
import csv
from glob import glob
from tf.fabric import Fabric
from tf.convert.walker import CV
from tf.compose import modify

First, get the data.

In [38]:
import csv
word_dict = {}
phr_dict = {}
cl_dict = {}

# process words
with open("C:/Users/geitb/Documents/CACCHT/DSS2ETCBC/lexemes_pos_all_bib_books.csv") as file: # Use file to refer to the file object

    reader = csv.reader(file, delimiter=",")
    isa_index = 1
    for line in reader:
        if line[1] == '1Qisaa':
            word_dict[isa_index] = line
            isa_index += 1
            

# process phrases
with open("C:/Users/geitb/Documents/CACCHT/DSS2ETCBC/_1_q_isaa_phrases.csv") as file: # Use file to refer to the file object

    reader = csv.reader(file, delimiter=",")
    for i, line in enumerate(reader):
        phr_dict[i] = line

# process clauses
with open("C:/Users/geitb/Documents/CACCHT/DSS2ETCBC/_1_q_isaa_clause_boundaries.csv") as file: # Use file to refer to the file object

    reader = csv.reader(file, delimiter=",")
    for i, line in enumerate(reader):
        cl_dict[i] = line

for i in range(len(phr_dict)):
    if phr_dict[i][2] != cl_dict[i][2]:
        print(phr_dict[i], cl_dict[i])

In [41]:
def director(cv):
        
    '''
    Walks through 1QIsaa, and creates 
    '''
        
    # process books in order
    #for bo, book in bo2book.items():
    # process words
    bo = '1QIsaa'
    book = 'Isaiah'
    
    this_book = cv.node('book')
    cv.feature(this_book, book='1QIsaa')

            
    # keep track of when to trigger clause, chapter, and verse objects
    clause_track = 1 # keep counts of clauses
    prev_chap = 1 # start at 1
    prev_verse = 1 # start at 1
    this_chap = cv.node('chapter')
    this_verse = cv.node('verse')
    this_clause = cv.node('clause')      
        
    clause_end = False
        
    # iterate through words and construct objects
    for word in range(1,len(word_dict)):
            
        data = word_dict[word]
        cl_info = cl_dict[word]
        
        # segment out word data
        g_cons = data[5]
        lex = data[7]
        pos = data[9]

        chapt, verse, wrdnum = data[3], data[4], data[0]        
            
            
        # detect chapter boundary
        if prev_chap != chapt:
                
            # end verse
            cv.feature(this_verse, verse=prev_verse)
            cv.terminate(this_verse)
                
            # end chapter
            cv.feature(this_chap, chapter=prev_chap)
            cv.terminate(this_chap)
                
            # new chapter and verse begin
            this_chap = cv.node('chapter')
            prev_chap = chapt
            this_verse = cv.node('verse')
            prev_verse = verse
            
        # detect verse boundary
        elif prev_verse != verse:
            cv.feature(this_verse, verse=prev_verse)
            cv.terminate(this_verse)
            this_verse = cv.node('verse') # start a new verse
            prev_verse = verse
                
        if clause_end == True:
            cv.feature(this_clause, clause=clause_track)
            cv.terminate(this_clause)
            this_clause = cv.node('clause') # start a new clause
            clause_track += 1 # count clauses
            clause_end = False
                
        # detect clause boundary
        if cl_info[4] == 'e':
            clause_end = True
                
                
        # make word object
        this_word = cv.slot()
        cv.feature(this_word, 
                       sp=pos, 
                       lex=lex, 
                       g_cons=g_cons, 
                      )
        cv.terminate(this_word)
        
    # - end clause
    cv.feature(this_clause, clause=clause_track)
    cv.terminate(this_clause)
        
    # - end verse
    cv.feature(this_verse, verse=prev_verse)
    cv.terminate(this_verse)
        
    # - end chapter
    cv.feature(this_chap, chapter=prev_chap)
    cv.terminate(this_chap)
        
    # - end book
    cv.feature(this_book, book=book, book_code=bo)
    cv.terminate(this_book)

In [42]:
slotType = 'word'
otext = {'fmt:text-orig-full':'{g_cons} ',
         'sectionTypes':'book,chapter,verse,clause',
         'sectionFeatures':'book,chapter,verse,clause'}

generic = {'Name': 'DSSA',
           'Version': 0.1, # to be filled in
           'Editors': 'Martin Abegg',
           'Converter': 'Martijn Naaijer', 
           'Source:':'https://github.com/ETCBC/dss',
           'Note':'in progress'}

intFeatures = {'chapter', 'verse'}

featureMeta = {'book': {'description': 'A biblical book name'},
               'book_code': {'description': 'manuscript code'},
               'chapter': {'description': 'A chapter number'},
               'verse': {'description': 'A verse number'},
               'clause': {'description': 'A clause number'},
               'lex': {'description': 'lexeme in etcbc transcription'},
               'g_cons': {'description': 'surface structure of a word'},
               'sp': {'description': 'part of speech'}
              }

In [43]:
output_dir = 'C:/Users/geitb/Documents/CACCHT/DSS2ETCBC/tf'

TF = Fabric(locations=output_dir, silent=True)
cv = CV(TF)
    
good = cv.walk(director,
                   slotType,
                   otext=otext,
                   generic=generic,
                   intFeatures=intFeatures,
                   featureMeta=featureMeta,
                   warn=True,
                   force=False,)

  0.00s Importing data from walking through the source ...
   |     0.00s Preparing metadata... 
   |     0.00s No structure nodes will be set up
   |   SECTION   TYPES:    book, chapter, verse, clause
   |   SECTION   FEATURES: book, chapter, verse, clause
   |   STRUCTURE TYPES:    
   |   STRUCTURE FEATURES: 
   |   TEXT      FEATURES:
   |      |   text-orig-full       g_cons
   |     0.00s OK
   |     0.00s Following director... 
   |     0.16s "edge" actions: 0
   |     0.16s "feature" actions: 29818
   |     0.16s "node" actions: 7045
   |     0.16s "resume" actions: 0
   |     0.16s "slot" actions: 22772
   |     0.16s "terminate" actions: 29817
   |          1 x "book" node 
   |         67 x "chapter" node 
   |       5686 x "clause" node 
   |       1291 x "verse" node 
   |      22772 x "word" node  = slot type
   |      29817 nodes of all types
   |     0.17s OK
   |     0.00s Removing unlinked nodes ... 
   |      |     0.00s      1 unlinked "chapter" node: [1]
   |      

  0.18s WARNING section chapter "1" of level 2 ends inside a clause "??" of level 4 (2 x):
   |      |     0.29s WARNING section chapter "10" of level 2 ends inside a clause "??" of level 4 (1 x):
   |      |     0.29s WARNING section chapter "11" of level 2 ends inside a clause "??" of level 4 (1 x):
   |      |     0.29s WARNING section chapter "12" of level 2 ends inside a clause "??" of level 4 (1 x):
   |      |     0.29s WARNING section chapter "13" of level 2 ends inside a clause "??" of level 4 (1 x):
   |      |     0.29s WARNING section chapter "14" of level 2 ends inside a clause "??" of level 4 (1 x):
   |      |     0.29s WARNING section chapter "15" of level 2 ends inside a clause "??" of level 4 (1 x):
   |      |     0.29s WARNING section chapter "16" of level 2 ends inside a clause "??" of level 4 (1 x):
   |      |     0.29s WARNING section chapter "17" of level 2 ends inside a clause "??" of level 4 (1 x):
   |      |     0.29s WARNING section chapter "18" of level 2

   |      |     0.34s WARNING section verse "20" of level 3 ends inside a clause "??" of level 4 (36 x):
   |      |     0.34s WARNING section verse "21" of level 3 ends inside a clause "??" of level 4 (34 x):
   |      |     0.34s WARNING section verse "22" of level 3 ends inside a clause "??" of level 4 (31 x):
   |      |     0.35s WARNING section verse "23" of level 3 ends inside a clause "??" of level 4 (25 x):
   |      |     0.35s WARNING section verse "24" of level 3 ends inside a clause "??" of level 4 (21 x):
   |      |     0.35s WARNING section verse "25" of level 3 ends inside a clause "??" of level 4 (19 x):
   |      |     0.35s WARNING section verse "26" of level 3 ends inside a clause "??" of level 4 (13 x):
   |      |     0.35s WARNING section verse "27" of level 3 ends inside a clause "??" of level 4 (11 x):
   |      |     0.35s WARNING section verse "28" of level 3 ends inside a clause "??" of level 4 (11 x):
   |      |     0.35s WARNING section verse "29" of lev

   |      |     0.36s use `cv.walk(..., warn=False)` to continue after warnings
   |      |     0.36s use `cv.walk(..., warn=None)` to suppress warnings


In [44]:
TF = Fabric(locations=output_dir)
api = TF.load('''

book chapter verse clause
g_cons lex sp


''')

classes = api.makeAvailableIn(globals())

This is Text-Fabric 7.9.0
Api reference : https://annotation.github.io/text-fabric/Api/Fabric/

11 features found and 0 ignored
  0.00s loading features ...
   |     0.02s T otype                from C:/Users/geitb/Documents/CACCHT/DSS2ETCBC/tf
   |     0.07s T oslots               from C:/Users/geitb/Documents/CACCHT/DSS2ETCBC/tf
   |     0.00s No section config in otext, the section part of the T-API cannot be used
   |     0.00s No structure info in otext, the structure part of the T-API cannot be used
   |     0.06s T g_cons               from C:/Users/geitb/Documents/CACCHT/DSS2ETCBC/tf
   |      |     0.01s C __levels__           from otype, oslots, otext
   |      |     0.14s C __order__            from otype, oslots, __levels__
   |      |     0.01s C __rank__             from otype, __order__
   |      |     0.15s C __levUp__            from otype, oslots, __rank__
   |      |     0.02s C __levDown__          from otype, __levUp__, __rank__
   |      |     0.07s C __boundary__

In [45]:
for w in F.otype.s('word'):
    ch = F.chapter.v(L.u(w, 'chapter')[0])
    ve = F.verse.v(L.u(w, 'verse')[0])
    print(w, ch, ve, F.sp.v(w), F.g_cons.v(w), F.lex.v(w))

1 1 1 subs XZWN XZWN/
2 1 1 nmpr JC<JHW JC<JHW/
3 1 1 subs BN BN/
4 1 1 nmpr >MWY >MWY/
5 1 1 conj >CR >CR
6 1 1 verb XZH XZH[
7 1 1 prep <L <L
8 1 1 nmpr JHWDH JHWDH/
9 1 1 conj W W
10 1 1 nmpr JRWCLM JRWCLM/
11 1 1 prep B B
12 1 1 subs JWMJ JWM/
13 1 1 nmpr <WZJH <ZJHW/
14 1 1 nmpr JWTM JWTM/
15 1 1 nmpr >XZ >XZ=/
16 1 1 nmpr JXZQJH JXZQJHW/
17 1 1 subs MLKJ MLK/
18 1 1 nmpr JHWDH JHWDH/
19 1 2 verb CM<W CM<[
20 1 2 subs CMJM CMJM/
21 1 2 conj W W
22 1 2 verb H>ZJNJ >ZN[
23 1 2 art H H
24 1 2 subs >RY >RY/
25 1 2 conj KJ> KJ
26 1 2 nmpr JHWH JHWH/
27 1 2 verb DBR DBR[
28 1 2 subs BNJM BN/
29 1 2 verb GDLTJ GDL[
30 1 2 conj W W
31 1 2 verb RWMMTJ RWM[
32 1 2 conj W W
33 1 2 prps HMH HM
34 1 2 verb PC<W PC<[
35 1 2 prep BJ B
36 1 3 verb JD< JD<[
37 1 3 subs CWR CWR/
38 1 3 verb QWNJHW QNH[
39 1 3 conj W W
40 1 3 subs XMWR XMWR/
41 1 3 subs >BWS >BWS/
42 1 3 subs B<LJW B<L/
43 1 3 nmpr JFR>L JFR>L/
44 1 3 nega LW> L>
45 1 3 verb JD< JD<[
46 1 3 conj W W
47 1 3 subs <MJ <M/
48 1 3 nega L

372 1 25 conj W W
373 1 25 verb HCJB CWB[
374 1 25 subs JDJ JD/
375 1 25 prep <LJK <L
376 1 25 conj W W
377 1 25 verb >YRP YRP[
378 1 25 prep K K
379 1 25 subs BR BR=====/
380 1 25 subs SWGJK SJG/
381 1 25 conj W W
382 1 25 verb >SJR SWR[
383 1 25 subs KWL KL/
384 1 25 subs BDJLJK BDJL/
385 1 26 conj W W
386 1 26 verb >CJBH CWB[
387 1 26 verb CWPVJK CPV[
388 1 26 prep K K
389 1 26 prep B B
390 1 26 adjv R>JCWNH R>CWN/
391 1 26 conj W W
392 1 26 verb J<YJK J<Y[
393 1 26 prep K K
394 1 26 prep B B
395 1 26 subs TXLH TXLH/
396 1 26 subs >XRJ >XR/
397 1 26 advb KN KN
398 1 26 verb JQR>W QR>[
399 1 26 prep LK L
400 1 26 subs <JR <JR/
401 1 26 art H H
402 1 26 subs YDQ YDQ/
403 1 26 subs QRJH QRJH/
404 1 26 verb N>MNH >MN[
405 1 27 nmpr YJWN YJWN==/
406 1 27 prep B B
407 1 27 subs MCPV MCPV/
408 1 27 verb TPDH PDH[
409 1 27 conj W W
410 1 27 verb CBJH CWB[
411 1 27 prep B B
412 1 27 subs YDQH YDQH/
413 1 28 conj W W
414 1 28 subs CBR CBR/
415 1 28 verb PWC<JM PC<[
416 1 28 conj W W
417 1 28 

817 2 22 subs NCMH NCMH/
818 2 22 prep B B
819 2 22 subs >PW >P/
820 2 22 conj KJ> KJ
821 2 22 prep B B
822 2 22 prin MH MH
823 2 22 verb NXCB XCB[
824 2 22 prps HW> HW>
825 3 1 conj KJ> KJ
826 3 1 intj HNH HNH
827 3 1 art H H
828 3 1 subs >DWN >DWN/
829 3 1 nmpr JHWH JHWH/
830 3 1 subs YB>WT YB>/
831 3 1 verb MHSJR SWR[
832 3 1 prep M MN
833 3 1 nmpr JRWCLM JRWCLM/
834 3 1 conj W W
835 3 1 prep M MN
836 3 1 nmpr JHWDH JHWDH/
837 3 1 subs MC<N MC<N/
838 3 1 conj W W
839 3 1 subs MC<NH MC<NH/
840 3 1 subs KL KL/
841 3 1 subs MC<N MC<N=/
842 3 1 subs LXM LXM/
843 3 1 conj W W
844 3 1 subs KWL KL/
845 3 1 subs MC<N MC<N=/
846 3 1 subs MJM MJM/
847 3 2 subs GBWR GBWR/
848 3 2 conj W W
849 3 2 subs >JC >JC/
850 3 2 subs MLXMH MLXMH/
851 3 2 verb CWPV CPV[
852 3 2 conj W W
853 3 2 subs NBJ> NBJ>/
854 3 2 conj W W
855 3 2 verb QWSM QSM[
856 3 2 conj W W
857 3 2 adjv ZQN ZQN/
858 3 3 subs FR FR/
859 3 3 subs XMCJM XMC/
860 3 3 conj W W
861 3 3 verb NF> NF>[
862 3 3 subs PNJM PNH/
863 3 3 conj 

1229 4 2 conj W W
1230 4 2 subs PRJ PRJ/
1231 4 2 art H H
1232 4 2 subs >RY >RY/
1233 4 2 prep L L
1234 4 2 subs G>WN G>WN/
1235 4 2 conj W W
1236 4 2 prep L L
1237 4 2 subs TP>RT TP>RT/
1238 4 2 prep L L
1239 4 2 subs PLJVT PLJVH/
1240 4 2 nmpr JFR>L JFR>L/
1241 4 2 conj W W
1242 4 2 nmpr JHWDH JHWDH/
1243 4 3 conj W W
1244 4 3 verb JHJH HJH[
1245 4 3 art H H
1246 4 3 verb NC>R C>R[
1247 4 3 prep B B
1248 4 3 nmpr YJWN YJWN==/
1249 4 3 conj W W
1250 4 3 art H H
1251 4 3 verb NWTR JTR[
1252 4 3 prep B B
1253 4 3 nmpr JRWCLM JRWCLM/
1254 4 3 adjv QDWC QDWC/
1255 4 3 verb J>MR >MR[
1256 4 3 prep LW L
1257 4 3 subs KWL KL/
1258 4 3 art H H
1259 4 3 verb KTWB KTB[
1260 4 3 prep L L
1261 4 3 subs XJJM XJJM/
1262 4 3 prep B B
1263 4 3 nmpr JRWCLM JRWCLM/
1264 4 4 conj >M >M
1265 4 4 verb RXY RXY[
1266 4 4 nmpr >DWNJ >DNJ/
1267 4 4 prep >T >T
1268 4 4 subs Y>T Y>H=/
1269 4 4 subs BNWT BT/
1270 4 4 nmpr YJWN YJWN==/
1271 4 4 conj W W
1272 4 4 prep >T >T
1273 4 4 subs DMJ DM/
1274 4 4 nmpr JRWC

1613 5 18 subs XV>H XV>H=/
1614 5 19 art H H
1615 5 19 verb >WMRJM >MR[
1616 5 19 verb JMHR MHR[
1617 5 19 verb JXJC XWC[
1618 5 19 subs M<FJHW M<FH/
1619 5 19 prep LM<N LM<N
1620 5 19 verb NR>H R>H[
1621 5 19 conj W W
1622 5 19 verb TQRBH QRB[
1623 5 19 conj W W
1624 5 19 verb TBW>H BW>[
1625 5 19 subs <YT <YH/
1626 5 19 adjv QDWC QDWC/
1627 5 19 nmpr JFR>L JFR>L/
1628 5 19 conj W W
1629 5 19 verb ND< JD<[
1630 5 20 intj HWJ HWJ
1631 5 20 art H H
1632 5 20 verb >WMRJM >MR[
1633 5 20 prep L L
1634 5 20 adjv R< R</
1635 5 20 adjv VWB VWB/
1636 5 20 conj W W
1637 5 20 prep L L
1638 5 20 adjv VWB VWB/
1639 5 20 adjv R< R</
1640 5 20 verb FMJM FJM[
1641 5 20 subs XWCK XCK/
1642 5 20 prep L L
1643 5 20 subs >WR >WR/
1644 5 20 conj W W
1645 5 20 subs >WR >WR/
1646 5 20 prep L L
1647 5 20 subs XWCK XCK/
1648 5 20 verb FMJM FJM[
1649 5 20 adjv MR MR/
1650 5 20 prep L L
1651 5 20 adjv MTWQ MTWQ/
1652 5 20 conj W W
1653 5 20 adjv MTWQ MTWQ/
1654 5 20 prep L L
1655 5 20 adjv MR MR/
1656 5 21 intj

...

20945 61 7 conj W W
20946 61 7 subs KLMH KLMH/
20947 61 7 verb JRWNW RNN[
20948 61 7 subs XLQKMH XLQ/
20949 61 7 advb L LKN
20950 61 7 advb KN LKN
20951 61 7 subs MCNH MCNH/
20952 61 7 prep B B
20953 61 7 subs >RYM >RY/
20954 61 7 verb TJRCW JRC[
20955 61 7 subs FMXT FMXH/
20956 61 7 subs <WLM <WLM/
20957 61 7 verb THJH HJH[
20958 61 7 prep LKMH L
20959 61 8 conj KJ> KJ
20960 61 8 prps >NJ >NJ
20961 61 8 nmpr JHWH JHWH/
20962 61 8 verb >WHB >HB[
20963 61 8 subs MCPV MCPV/
20964 61 8 conj W W
20965 61 8 verb FWNH FN>[
20966 61 8 subs GZWL GZL/
20967 61 8 prep B B
20968 61 8 subs <WLH <WLH/
20969 61 8 conj W W
20970 61 8 verb NTTJ NTN[
20971 61 8 subs P<WLTKM P<LH/
20972 61 8 prep B B
20973 61 8 subs >MT >MT/
20974 61 8 conj W W
20975 61 8 subs BRJT BRJT/
20976 61 8 subs <WLM <WLM/
20977 61 8 verb >KRWT KRT[
20978 61 8 prep LKMH L
20979 61 9 conj W W
20980 61 9 verb NWD< JD<[
20981 61 9 prep B B
20982 61 9 subs GW>JM GWJ/
20983 61 9 subs ZR<KMH ZR</
20984 61 9 conj W W
20985 61 9 subs Y>

21308 63 3 prep L L
21309 63 3 subs BDJ BD/
21310 63 3 conj W W
21311 63 3 prep M MN
21312 63 3 subs <MJ <M/
21313 63 3 subs >JN >JN/
21314 63 3 subs >JC >JC/
21315 63 3 prep >TJ >T==
21316 63 3 conj W W
21317 63 3 subs KWL KL/
21318 63 3 subs MLBWCJ MLBWC/
21319 63 3 verb G>LTJ G>L=[
21320 63 4 conj KJ> KJ
21321 63 4 subs JWM JWM/
21322 63 4 subs NQM NQM/
21323 63 4 prep B B
21324 63 4 subs LBJ LB/
21325 63 4 conj W W
21326 63 4 subs CNT CNH/
21327 63 4 subs G>WLJ G>WLJM/
21328 63 4 verb B>H BW>[
21329 63 5 conj W W
21330 63 5 verb >BJV NBV[
21331 63 5 conj W W
21332 63 5 subs >JN >JN/
21333 63 5 verb <WZR <ZR[
21334 63 5 conj W W
21335 63 5 verb >CTWMM CMM[
21336 63 5 conj W W
21337 63 5 subs >JN >JN/
21338 63 5 verb TWMK SMK[
21339 63 5 conj W W
21340 63 5 verb TWC< JC<[
21341 63 5 prep LJ> L
21342 63 5 subs ZRW<J ZRW</
21343 63 5 conj W W
21344 63 5 subs XMTJ> XMH/
21345 63 5 prps HJ> HJ>
21346 63 5 verb SMKTNJ SMK[
21347 63 6 conj W W
21348 63 6 verb >BWSH BWS[
21349 63 6 subs <MJ

21749 64 11 subs MW>DH M>D/
21750 65 1 verb NDRCTJ DRC[
21751 65 1 prep L L
21752 65 1 nega LW> L>
21753 65 1 verb C>LWNJ C>L[
21754 65 1 verb NMYJTJ MY>[
21755 65 1 prep L L
21756 65 1 nega LW> L>
21757 65 1 verb BQCWNJ BQC[
21758 65 1 verb >MRTJ >MR[
21759 65 1 intj HNNJ HNH
21760 65 1 intj HNNJ HNH
21761 65 1 prep >L >L
21762 65 1 subs GWJ GWJ/
21763 65 1 nega LW> L>
21764 65 1 verb QR> QR>[
21765 65 1 prep B B
21766 65 1 subs CMJ> CM/
21767 65 2 verb PRFTJ PRF[
21768 65 2 subs JDJ JD/
21769 65 2 subs KWL KL/
21770 65 2 art H H
21771 65 2 subs JWM JWM/
21772 65 2 prep >L >L
21773 65 2 subs <M <M/
21774 65 2 verb SWRH SRR[
21775 65 2 art H H
21776 65 2 verb HWLKJM HLK[
21777 65 2 art H H
21778 65 2 subs DRK DRK/
21779 65 2 nega LW> L>
21780 65 2 adjv VWB VWB/
21781 65 2 subs >XR >XR/
21782 65 2 subs MXCBWTJHMH MXCBH/
21783 65 3 art H H
21784 65 3 subs <M <M/
21785 65 3 art H H
21786 65 3 verb MK<JSJM K<S[
21787 65 3 prep >WTJ >T
21788 65 3 prep <L <L
21789 65 3 subs PNJ PNH/
21790 65

22102 65 19 conj W W
22103 65 19 verb FFTJ FWF[
22104 65 19 prep B B
22105 65 19 subs <MJ> <M/
22106 65 19 conj W W
22107 65 19 nega LW> L>
22108 65 19 verb JCM< CM<[
22109 65 19 prep BH B
22110 65 19 subs <WD <WD/
22111 65 19 subs QWL QWL/
22112 65 19 subs BKJ BKJ/
22113 65 19 conj W W
22114 65 19 subs QWL QWL/
22115 65 19 subs Z<QH Z<QH/
22116 65 20 conj W W
22117 65 20 nega LW> L>
22118 65 20 verb JHJH HJH[
22119 65 20 prep M MN
22120 65 20 advb CMH CM
22121 65 20 subs <WD <WD/
22122 65 20 subs <WJL <WL/
22123 65 20 subs JMJM JWM/
22124 65 20 conj W W
22125 65 20 adjv ZQN ZQN/
22126 65 20 conj >CR >CR
22127 65 20 nega LW> L>
22128 65 20 verb JMLH ML>[
22129 65 20 prep >T >T
22130 65 20 subs JMJW JWM/
22131 65 20 conj KJ> KJ
22132 65 20 art H H
22133 65 20 subs N<R N<R/
22134 65 20 subs BN BN/
22135 65 20 subs M>H M>H/
22136 65 20 subs CNH CNH/
22137 65 20 verb JMWT MWT[
22138 65 20 conj W W
22139 65 20 art H H
22140 65 20 verb XWV> XV>[
22141 65 20 subs BN BN/
22142 65 20 subs M>H M

22553 66 16 conj W W
22554 66 16 verb RBW RBB[
22555 66 16 adjv XLLJW XLL/
22556 66 17 art H H
22557 66 17 verb MTQDCJM QDC[
22558 66 17 conj W W
22559 66 17 art H H
22560 66 17 verb MVHRJM VHR[
22561 66 17 prep >L >L
22562 66 17 art H H
22563 66 17 subs GNWT GNH/
22564 66 17 subs >XR >XR/
22565 66 17 subs >XT >XD/
22566 66 17 prep B B
22567 66 17 subs TWK TWK/
22568 66 17 verb >WKLJ >KL[
22569 66 17 subs BFR BFR/
22570 66 17 art H H
22571 66 17 subs XWZJR XZJR/
22572 66 17 conj W W
22573 66 17 art H H
22574 66 17 subs CQWY CQY/
22575 66 17 conj W W
22576 66 17 art H H
22577 66 17 subs <KBR <KBR/
22578 66 17 advb JXDJW JXDW
22579 66 17 subs >MR N>M/
22580 66 17 nmpr JHWH JHWH/
22581 66 18 conj W W
22582 66 18 prps >NWKJ >NKJ
22583 66 18 subs M<FJHMH M<FH/
22584 66 18 conj W W
22585 66 18 subs MXCBWTJHMH MXCBH/
22586 66 18 verb B>W BW>[
22587 66 18 prep L L
22588 66 18 verb QBY QBY[
22589 66 18 prep >T >T
22590 66 18 subs KWL KL/
22591 66 18 art H H
22592 66 18 subs GW>JM GWJ/
22593 66 

In [46]:
for cl in F.otype.s('clause'):
    words = L.d(cl, 'word')
    lexemes = [F.lex.v(w) for w in words]
    print(lexemes)

['XZWN/', 'JC<JHW/', 'BN/', '>MWY/']
['>CR', 'XZH[', '<L', 'JHWDH/', 'W', 'JRWCLM/', 'B', 'JWM/', '<ZJHW/', 'JWTM/', '>XZ=/', 'JXZQJHW/', 'MLK/', 'JHWDH/']
['CM<[']
['CMJM/']
['W', '>ZN[']
['H', '>RY/']
['KJ', 'JHWH/', 'DBR[']
['BN/', 'GDL[']
['W', 'RWM[']
['W', 'HM', 'PC<[', 'B']
['JD<[', 'CWR/', 'QNH[']
['W', 'XMWR/', '>BWS/', 'B<L/']
['JFR>L/', 'L>', 'JD<[']
['W', '<M/', 'L>', 'BJN[']
['HWJ', 'GWJ/', 'XV>[', '<M/', 'KBD/', '<WN/', 'ZR</', 'R<<[', 'BN/', 'CXT[']
['<ZB[', '>T', 'JHWH/']
['N>Y[', '>T', 'QDWC/', 'JFR>L/']
['ZWR[', '>XWR/']
['<L', 'MH', 'NKH[', '<WD/']
['JSP[', 'SRH/']
['KL/', 'R>C/', 'L', 'XLJ/']
['W', 'KL/', 'LBB/', 'DWJ=/']
['MN', 'KP/', 'RGL/', 'W', '<D', 'R>C/', '>JN/', 'B', 'MTM/']
['PY</', 'W', 'XBWRH/', 'W', 'MKH/', 'VRJ/']
['L>', 'ZRH[']
['W', 'L>', 'XBC[']
['W', 'L>', 'RKK[', 'B', 'CMN/']
['>RY/', 'CMMH/']
['<JR/', 'FRP[', '>C/']
['>DMH/']
['L', 'NGD/', 'ZR/', '>KL[', '>T']
['W', 'CMMH/', '<L', 'K', 'MHPKH/', 'ZR/']
['W', 'JTR[', 'BT/', 'YJWN==/', 'K', 'SKH/', 

['<L', 'NBW=/', 'W', '<L', 'MJDB>/', 'MW>B/', 'JLL[']
['B', 'KL/', 'R>C/', 'QRXH/']
['W', 'KL/', 'ZQN=/', 'GR<[']
['B', 'XWY/', 'XGR[', 'FQ/']
['<L', 'GG/', 'W', 'B', 'RXB==/', 'KL/', 'JLL[']
['W', 'JRD[', 'B', 'BKJ/']
['W', 'Z<Q[', 'XCBWN==/', 'W', '>L<LH/']
['<D', 'JHY/', 'CM<[', 'QWL/']
['<L', 'KN', 'XLY[', 'MW>B/', 'RW<[']
['NPC/', 'JR<[', 'L']
['LB/', 'L', 'MW>B/', 'Z<Q[']
['BRJX/', '<D', 'Y<R/', '<GLT_CLCJH/', '<GLT_CLCJH/']
['KJ', 'M<LH/', 'H', 'LWXJT/']
['B', 'BKJ/', '<LH[', 'B']
['KJ', 'DRK/', 'XRWNJM/', 'Z<QH/', 'CBR/', '<WR[']
['KJ', 'MJM/', 'NMRJM/', 'MCMH/', 'HJH[']
['KJ', 'JBC[', 'XYJR/']
['KLH[', 'DC>/']
['JRQ/', 'L>', 'HJH[']
['<L', 'KN', 'JTRH/', '<FH[']
['W', 'PQDH/']
['<L', 'NXL/', '<RBH=/', 'NF>[']
['KJ', 'NQP[', 'H', 'Z<QH/', '>T', 'GBWL/', 'MW>B/']
['<D', '>GLJM/', 'JLLH/']
['W', 'B>R_>JLJM/', 'B>R_>JLJM/', 'JLLH/']
['KJ', 'MJM/', 'DJMWN/', 'ML>[', 'DM/']
['KJ', 'CJT[', '<L', 'DJMWN/', 'JSP[']
['L', 'PLJVH/', 'MW>B/', '>RJH/']
['L', 'C>RJT/', '>DMH/']
['CLX[', 'KR

['W', 'H', 'KRML/', 'L', 'J<R/', 'XCB[']
['W', 'CM<[', 'B', 'JWM/', 'H', 'HW>', 'H', 'XRC=/', 'DBR/', 'SPR/']
['W', 'MN', '>PL=/', 'W', 'MN', 'XCK/', '<JN/', '<WR/', 'R>H[']
['W', 'JSP[', '<NW/', 'B', 'JHWH/', 'FMXH/']
['W', '>BJWN/', '>DM/', 'B', 'QDWC/', 'JFR>L/', 'GJL[']
['KJ', '>PS[', '<RJY/']
['W', 'KLH[', 'LY/']
['W', 'KRT[', 'KL/', 'CQD[', '>WN=/']
['XV>[', '>DM/', 'B', 'DBR/']
['W', 'L', 'JKX[', 'B', 'C<R/', 'QWC[']
['W', 'NVH[', 'B', 'THW/', 'YDJQ/']
['LKN', 'LKN']
['KH', '>MR[', 'JHWH/', '>L', 'BJT/', 'J<QB/']
['>CR', 'PDH[', '>T', '>BRHM/']
['L>', '<TH', 'BWC[', 'J<QB/']
['W', 'L>', '<TH', 'PNH/', 'XWR[']
['KJ', 'B', 'R>H[', 'JLD/', 'M<FH/', 'JD/', 'B', 'QRB/']
['QDC[', 'CM/']
['W', 'QDC[', '>T', 'QDWC/', 'J<QB/']
['W', '>T', '>LHJM/', 'JFR>L/', '<RY[']
['W', 'JD<[', 'T<H[', 'RWX/', 'BJNH/']
['W', 'RGN[', 'LMD[', 'LQX/']
['HWJ', 'BN/']
['SRR[']
['N>M/', 'JHWH/']
['L', '<FH[', '<YH/']
['W', 'L>', 'MN']
['W', 'L', 'NSK[', 'MSKH==/']
['W', 'L>', 'RWX/']
['LM<N', 'SPH[', 'XV>T/'

['W', 'KJ', '>MR[', '>L']
['<L', 'JHWH/', '>LHJM/', 'BVX[']
['H=', 'L>', 'HW>']
['>CR', 'SWR[', 'XZQJHW/', '>T', 'BMH/', 'W', '>T', 'MZBX/']
['W', '>MR[', 'L', 'JHWDH/', 'W', 'L', 'JRWCLM/']
['L', 'PNH/', 'H', 'MZBX/', 'H', 'ZH', 'XWH[']
['B', 'JRWCLM/']
['W', '<TH']
['<RB[', 'N>', '>T==', '>DWN/', 'H', 'MLK/', '>CWR/']
['W', 'NTN[', 'L', '>LP=/', 'SWS/']
['>M', 'JKL[']
['L', 'NTN[', 'L', 'RKB[', '<L']
['W', '>JK', 'CWB[', '>T', 'PNH/', 'PXH/', '>XD/', 'MN', '<BD/', '>DWN/', 'H', 'QVN=/']
['W', 'BVX[', 'L', '<L', 'MYRJM/', 'L', 'RKB/', 'W', 'L', 'PRC/']
['W', '<TH']
['H=', 'MN', 'BL<DJ', 'JHWH/', '<LH[', '<L', 'H', '>RY/', 'H', 'Z>T']
['L', 'CXT[']
['JHWH/', '>MR[', '>L']
['<LH[', '>L', 'H', '>RY/', 'H', 'Z>T']
['L', 'CXT[']
['W', '>MR[', '>LJQJM/', '>LJQJM/', 'W', 'CBN>/', 'W', 'JW>X/']
['DBR[', 'N>', '<M', '<BD/', '<M', '>RMJ=/']
['KJ', 'CM<[', '>NXNW']
['W', '>L=', 'DBR[', '>T', 'JHWDJ/', 'DBR/', 'H', '>LH', 'B', '>ZN/', 'H', '>JC/']
['H', 'JCB[', '<L', 'H', 'XWMH/']
['W', '>MR[', '

['W', 'RWX/']
['L', 'HLK[', 'B']
['>NJ', 'JHWH/', 'QR>[', 'B', 'YDQ/']
['W', 'XZQ[', 'B', 'JD/']
['W', 'NYR[']
['W', 'NTN[', 'L', 'BRJT/', '<M/', 'L', '>WR/', 'GWJ/']
['L', 'PQX[', '<JN/', '<WR/']
['L', 'JY>[', 'MN', 'MSGR/', '>SJR=/']
['W', 'MN', 'BJT/', 'KL>/', 'JCB[', 'XCK/']
['>NJ', 'JHWH/']
['HW>', 'W', 'CM/']
['W', 'KBWD/', 'L', '>XR=/', 'L>', 'NTN[']
['W', 'THLH/', 'L', 'PSJL/']
['H', 'R>CWN/']
['HNH', 'BW>[']
['W', 'H', 'XDC/', '>NJ', 'NGD[']
['B', 'VRM/', 'YMX[']
['CM<[', '>T']
['CJR[', 'L', 'JHWH/', 'CJR/', 'XDC/']
['W', 'THLH/', 'MN', 'QYH=/', 'H', '>RY/']
['JRD[', 'H', 'JM/', 'W', 'ML>=/']
['>J/', 'W', 'JCB[']
['NF>[', 'MDBR/', '<JR/', 'W', 'XYR/']
['JCB[', 'QDR/']
['W', 'RNN[', 'JCB[', 'SL</']
['MN', 'R>C/', 'HR/', 'YWX[']
['FJM[', 'L', 'JHWH/', 'KBWD/']
['W', 'THLH/', 'B', '>J/', 'NGD[']
['JHWH/', 'K', 'GBWR/', 'JY>[']
['K', '>JC/', 'MLXMH/', '<WR[', 'QN>H/']
['RW<[']
['>P', 'YRX[']
['<L', '>JB[', 'GBR[']
['XCH[', '>K', 'MN', '<WLM/']
['XRC[']
['>PQ[']
['K', 'JLD[', 'P<H[

['W', 'R>H[']
['KL/', 'QBY[']
['BW>[', 'L']
['XJ/', '>NJ']
['N>M/', 'JHWH/']
['KJ', 'KL/', 'K', '<DJ/', 'LBC[']
['W', 'QCR[', 'K', 'KLH/']
['KJ', 'XRBH=/', 'W', 'CMM[', 'W', '>RY/', 'HRJST/']
['KJ', '<TH', 'YRR[', 'MN', 'JCB[']
['W', 'RXQ[', 'BL<[']
['<WD/', '>MR[', 'B', '>ZN/', 'BN/', 'CKLJM/']
['YR/', 'L', 'H', 'MQWM/']
['NGC[', 'L']
['W', 'JCB[']
['W', '>MR[', 'B', 'LBB/']
['MJ', 'JLD[', 'L', '>T', '>LH']
['W', '>NJ', 'CKWL==/', 'W', 'GLMWD/']
['W', 'GLH[']
['W', 'SWR/']
['>LH']
['MJ', 'GDL[']
['HN', '>NJ', 'C>R[', 'L', 'BD/']
['>LH']
['>JPH', 'HM']
['KJ', 'KH', '>MR[', 'JHWH/']
['HNH', 'NF>[', '>L', 'GWJ/', 'JD/']
['W', '>L', 'H', '<M/', 'RWM[', 'NS/']
['W', 'BW>[', 'BN/', 'B', 'XYN/']
['W', 'BT/', '<L', 'KTP/', 'NF>[']
['W', 'HJH[', 'MLK/', '>MN[']
['W', 'FRH/', 'MJNQT/']
['>P/', '>RY/']
['XWH[', 'L']
['W', '<PR/', 'RGL/', 'LXK[']
['W', 'JD<[']
['KJ', '>NJ', 'JHWH/']
['>CR', 'L>', 'BWC[', 'QWH[']
['H=', 'LQX[', 'MN', 'GBWR/', 'MLQWX/']
['>M', 'CBJ/', '<RJY/', 'MLV[']
['KJ', 'KH', 

['>L=', 'XFK[']
['K', 'CWPR/', 'RWM[', 'QWL/']
['W', 'NGD[', 'L', '<M/', 'PC</']
['W', 'L', 'BJT/', 'J<QB/', 'XV>T/']
['>T', 'JWM/', 'W', 'JWM/', 'DRC[']
['W', 'D<T/', 'DRK/', 'XPY[', 'K', 'GWJ/']
['>CR', 'YDQH/', '<FH[']
['W', 'MCPV/', '>LHJM/', 'L>', '<ZB[']
['C>L[', 'MCPV/', 'YDQ/']
['QRBH/', '>LHJM/', 'XPY[']
['LMH', 'YWM[']
['W', 'L>', 'R>H[']
['<NH=[', 'NPC/']
['W', 'L>', 'JD<[']
['HN', 'B', 'JWM/', 'YWM/', 'MY>[', 'XPY/']
['W', 'KL/', '<YB=====/', 'NGF[']
['HN', 'L', 'RJB/', 'W', 'L', 'MYH=/', 'YWM[']
['W', 'L', 'NKH[', 'B', '>GRP/', 'RC<=/']
['L>', 'YWM[', 'K', 'JWM/']
['L', 'CM<[', 'B', 'MRWM/', 'QWL/']
['H=', 'K', 'ZH', 'HJH[', 'YWM/']
['BXR[']
['JWM/']
['<NH=[', '>DM/', 'NPC/']
['H=', 'L', 'KPP[', 'K', '>GMWN/', 'R>C/']
['FQ/', 'W', '>PR/', 'JY<[']
['H=', 'L', 'ZH', 'QR>[', 'YWM/', 'JWM/', 'RYWN/', 'L', 'JHWH/']
['H=', 'L>', 'ZH', 'H', 'YWM/']
['>CR', 'BXR[']
['PTX[', 'XRYB/', 'RC<=/']
['W', 'NTR[', '>GDH/', 'MWVH/']
['W', 'CLX[', 'RYY[', 'XPCJ/']
['W', 'KL/', 'MWVH/', 'NTQ[